## Сборка датасета (коды вместо описаний концепций; делю не по визитам, а по возрасту пациентов)

Данные взяты из Synthea Dataset - EHR, использую 'conditions.csv' с результатами посещения врачей (диагнозы, временные метки) и 'patients.csv' с информацией о пациентах (дата рождения)

Сгенерировала выборку из 10000 человек с параметрами по умолчанию (seed=43, размер и параметры можно задать другими)

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
path = "../synthea/conditions.csv"
data = pd.read_csv(path)

Всего присутствует 280 концепций:

In [3]:
len(data['DESCRIPTION'].unique())

280

Кроме диагнозов встречаются концепции, которые обозначают ситуации (например, 'ожидает пересадки почки') и обнаруженные факты ('безработный', 'вовлечён в деятельность, связанную с риском'),  их (not_deseases) из общего списка я удалила

In [4]:
# concepts = set(data['DESCRIPTION'])
# situations = set()

# for c in concepts:
#     if c.find('situation')!=-1:
#         situations.add(c)
        
# print(situations)

In [5]:
concepts = set(data['DESCRIPTION'])
disorders = set()

for c in concepts:
    if c.find('disorder')!=-1:
        disorders.add(c)

In [6]:
# not_deseases = {'Medication review due (situation)', 
#                 'Received higher education (finding)', 
#                 'Unemployed (finding)',
#                 'Part-time employment (finding)',
#                 'Full-time employment (finding)',
#                 'Received certificate of high school equivalency (finding)'}

data = data[data['DESCRIPTION'].isin(disorders)]
data.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
1,2014-02-07,2014-02-19,045937c3-b043-1c0c-e6b1-a2b94b465dd2,6c96593e-74c1-1361-c56a-eed1f4ea9af8,195662009,Acute viral pharyngitis (disorder)
6,2019-09-08,2019-09-20,045937c3-b043-1c0c-e6b1-a2b94b465dd2,2b087075-f9ad-71c2-820c-968fa6c29336,10509002,Acute bronchitis (disorder)
9,2021-02-07,2021-03-06,045937c3-b043-1c0c-e6b1-a2b94b465dd2,981c2bfe-4d2a-b223-67d1-13f11ff2d887,444814009,Viral sinusitis (disorder)
12,2015-07-05,2015-07-27,5509bed5-d067-5968-6a46-9ddc627d0919,71aac2b5-39a3-d8eb-3feb-ec40c98954a7,444814009,Viral sinusitis (disorder)
13,2016-03-06,2016-03-16,5509bed5-d067-5968-6a46-9ddc627d0919,93009227-a804-5826-bfd4-a7951b92e3eb,195662009,Acute viral pharyngitis (disorder)


In [7]:
data.shape

(70415, 6)

Как и авторы оригинальной статьи ([BEHRT: Transformer for Electronic Health Records](https://arxiv.org/pdf/1907.09538.pdf)), оставляю только тех пациентов, на которых есть хотя бы 5 записей (минимальное количество, чтобы можно было делать какие-то прогнозы) 

In [8]:
u = data.groupby('PATIENT').count()['CODE']>=5
data = data[data['PATIENT'].isin(u[u==True].keys())]
data.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
58,2015-03-22,2015-03-30,5eb0459d-b43c-be02-9ef0-e156e2077c91,6a2ed7c6-154e-7a2f-510b-eeecbdf6aaed,195662009,Acute viral pharyngitis (disorder)
61,2016-07-19,2016-08-03,5eb0459d-b43c-be02-9ef0-e156e2077c91,25a22618-9d0a-7b71-6013-07ffc2c72fd6,10509002,Acute bronchitis (disorder)
71,2019-12-18,2020-01-03,5eb0459d-b43c-be02-9ef0-e156e2077c91,ba51fa57-da42-cdac-6aef-5608e7668f01,444814009,Viral sinusitis (disorder)
75,2021-11-18,2021-12-02,5eb0459d-b43c-be02-9ef0-e156e2077c91,22386ef8-1936-3209-a0f9-043e7fcb4b3a,312608009,Laceration - injury (disorder)
77,2022-01-31,2022-02-01,5eb0459d-b43c-be02-9ef0-e156e2077c91,22386ef8-1936-3209-a0f9-043e7fcb4b3a,307426000,Acute infective cystitis (disorder)


В таблице с пациентами интересуют только колонки с индивидуальным идентификатором и датой рождения

In [9]:
path2 = "../synthea/patients.csv"
patients = pd.read_csv(path2)

In [10]:
patients = patients.rename(columns={'Id': 'PATIENT'})
patients = patients[['PATIENT', 'BIRTHDATE', 'GENDER']]
patients.head()

,PATIENT,BIRTHDATE,GENDER
0,045937c3-b043-1c0c-e6b1-a2b94b465dd2,2009-12-28,F
1,5509bed5-d067-5968-6a46-9ddc627d0919,2000-03-26,F
2,b59b2212-f90e-7614-d5e1-84d8164d74f3,1967-06-26,M
3,5eb0459d-b43c-be02-9ef0-e156e2077c91,2001-01-30,F
4,a0e778e9-cc65-09c5-fefb-09bbb5261925,1962-07-30,M


Объединяю информацию из таблиц patients и data в одну таблицу

In [11]:
data = data.merge(patients, on='PATIENT')[['START', 'PATIENT', 'CODE', 'BIRTHDATE', 'GENDER']]
data.head()

,START,PATIENT,CODE,BIRTHDATE,GENDER
0,2015-03-22,5eb0459d-b43c-be02-9ef0-e156e2077c91,195662009,2001-01-30,F
1,2016-07-19,5eb0459d-b43c-be02-9ef0-e156e2077c91,10509002,2001-01-30,F
2,2019-12-18,5eb0459d-b43c-be02-9ef0-e156e2077c91,444814009,2001-01-30,F
3,2021-11-18,5eb0459d-b43c-be02-9ef0-e156e2077c91,312608009,2001-01-30,F
4,2022-01-31,5eb0459d-b43c-be02-9ef0-e156e2077c91,307426000,2001-01-30,F


In [12]:
data[data['PATIENT']=="5eb0459d-b43c-be02-9ef0-e156e2077c91"]

,START,PATIENT,CODE,BIRTHDATE,GENDER
0,2015-03-22,5eb0459d-b43c-be02-9ef0-e156e2077c91,195662009,2001-01-30,F
1,2016-07-19,5eb0459d-b43c-be02-9ef0-e156e2077c91,10509002,2001-01-30,F
2,2019-12-18,5eb0459d-b43c-be02-9ef0-e156e2077c91,444814009,2001-01-30,F
3,2021-11-18,5eb0459d-b43c-be02-9ef0-e156e2077c91,312608009,2001-01-30,F
4,2022-01-31,5eb0459d-b43c-be02-9ef0-e156e2077c91,307426000,2001-01-30,F
5,2022-10-21,5eb0459d-b43c-be02-9ef0-e156e2077c91,195662009,2001-01-30,F


Колонки с датами перевожу в datetime-формат, чтобы затем посчитать количество прошедших лет (возраст пациента на момент события)

In [13]:
#!pip install python-dateutil

In [14]:
from dateutil.relativedelta import relativedelta

In [15]:
data['START'] = data['START'].apply(pd.to_datetime)
data['BIRTHDATE'] = data['BIRTHDATE'].apply(pd.to_datetime)

In [16]:
data = data.assign(AGE=[relativedelta(data['START'][i], data['BIRTHDATE'][i]).years for i in range(data.shape[0])])

In [17]:
data['SEX'] = data['GENDER'].apply(lambda g: 'MALE' if g=='M' else 'FEMALE')

In [18]:
data = data.drop(columns='GENDER')
data

,START,PATIENT,CODE,BIRTHDATE,AGE,SEX
0,2015-03-22,5eb0459d-b43c-be02-9ef0-e156e2077c91,195662009,2001-01-30,14,FEMALE
1,2016-07-19,5eb0459d-b43c-be02-9ef0-e156e2077c91,10509002,2001-01-30,15,FEMALE
2,2019-12-18,5eb0459d-b43c-be02-9ef0-e156e2077c91,444814009,2001-01-30,18,FEMALE
3,2021-11-18,5eb0459d-b43c-be02-9ef0-e156e2077c91,312608009,2001-01-30,20,FEMALE
4,2022-01-31,5eb0459d-b43c-be02-9ef0-e156e2077c91,307426000,2001-01-30,21,FEMALE
...,...,...,...,...,...,...
57035,2013-01-24,b6abcd53-87ec-be6a-c25f-adb88176b4d1,59621000,1963-01-27,49,FEMALE
57036,2016-01-01,b6abcd53-87ec-be6a-c25f-adb88176b4d1,125605004,1963-01-27,52,FEMALE
57037,2016-02-14,b6abcd53-87ec-be6a-c25f-adb88176b4d1,237602007,1963-01-27,53,FEMALE
57038,2017-04-02,b6abcd53-87ec-be6a-c25f-adb88176b4d1,444814009,1963-01-27,54,FEMALE


Получается таблица "пациент-код_концепции-дата_визита-пол-возраст"

In [19]:
data = data[['PATIENT', 'CODE', 'START', 'SEX', 'AGE']]
data.head()

,PATIENT,CODE,START,SEX,AGE
0,5eb0459d-b43c-be02-9ef0-e156e2077c91,195662009,2015-03-22,FEMALE,14
1,5eb0459d-b43c-be02-9ef0-e156e2077c91,10509002,2016-07-19,FEMALE,15
2,5eb0459d-b43c-be02-9ef0-e156e2077c91,444814009,2019-12-18,FEMALE,18
3,5eb0459d-b43c-be02-9ef0-e156e2077c91,312608009,2021-11-18,FEMALE,20
4,5eb0459d-b43c-be02-9ef0-e156e2077c91,307426000,2022-01-31,FEMALE,21


Группирую записи сначала по пациенту, затем по возрасту, после -- по дате  

In [23]:
gb = data.groupby(['PATIENT', 'AGE'])

Сохраняю в виде словаря (ключи -- пациенты, значения -- список из последовательностей концепций по визитам и список соответствующих значений возраста)

In [29]:
t = ('00024e94-ccaa-8554-8a4a-a6e25565db69', 22)
list(data.iloc[gb.indices.get(t)]['CODE'])

[307426000]

In [30]:
d = {}

for t in gb.indices.keys():
    if t[0] not in d:
        d[t[0]] = {}
        d[t[0]]['input'] = []
        d[t[0]]['age'] = []
        d[t[0]]['input_for_tokenizer'] = []
    
    l = list(data.iloc[gb.indices.get(t)]['CODE'])
    d[t[0]]['input'] = np.concatenate((d[t[0]]['input'], l, ['[SEP]']))
    d[t[0]]['age'] = np.concatenate((d[t[0]]['age'], [str(t[1])]*(len(l)+1)))
    d[t[0]]['input_for_tokenizer'] = np.concatenate((d[t[0]]['input_for_tokenizer'], l), dtype=str)

In [31]:
d['00024e94-ccaa-8554-8a4a-a6e25565db69']

{'input': array(['307426000', '[SEP]', '271737000', '[SEP]', '195662009',
        '195662009', '[SEP]', '195662009', '444814009', '[SEP]',
        '444814009', '[SEP]'], dtype='<U32'),
 'age': array(['22', '22', '26', '26', '28', '28', '28', '29', '29', '29', '30',
        '30'], dtype='<U32'),
 'input_for_tokenizer': array(['307426000', '271737000', '195662009', '195662009', '195662009',
        '444814009', '444814009'], dtype='<U32')}

Убираю лишний '\[SEP\]' в конце для input (для возраста соответственно)

In [32]:
for k in d.keys():
    d[k]['input'] = list(d[k]['input'][:-1])
    d[k]['age'] = list(np.concatenate(([d[k]['age'][0]], d[k]['age'][:-1])))
    d[k]['input_for_tokenizer'] = list(d[k]['input_for_tokenizer'])

In [33]:
d['00024e94-ccaa-8554-8a4a-a6e25565db69']

{'input': ['307426000',
  '[SEP]',
  '271737000',
  '[SEP]',
  '195662009',
  '195662009',
  '[SEP]',
  '195662009',
  '444814009',
  '[SEP]',
  '444814009'],
 'age': ['22',
  '22',
  '22',
  '26',
  '26',
  '28',
  '28',
  '28',
  '29',
  '29',
  '29',
  '30'],
 'input_for_tokenizer': ['307426000',
  '271737000',
  '195662009',
  '195662009',
  '195662009',
  '444814009',
  '444814009']}

In [35]:
with open('input_6.txt', 'w') as outfile:
    for k in d.keys():
        outfile.write(' '.join(d[k]['input']))
        outfile.write('\n')

In [36]:
with open('input_for_tokenizer_6.txt', 'w') as outfile:
    for k in d.keys():
        outfile.write(' '.join(d[k]['input_for_tokenizer']))
        outfile.write('\n')

### Распределение диагнозов в датасете

In [24]:
codes_list = data['CODE']
codes_list

In [28]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

weights = compute_class_weight('balanced', 
                     classes=np.unique(codes_list), 
                     y=np.array(codes_list))
len(weights)

118

In [29]:
weights

array([2.01412429e+01, 4.27778611e+00, 1.05774580e+00, 1.21760663e-01,
       9.66779661e+01, 2.06576851e+00, 1.85919166e+00, 2.30185634e+01,
       9.06922759e-01, 9.55315871e-01, 2.43766934e-01, 2.84346959e+01,
       4.74842663e-01, 5.96777569e-01, 1.35783660e+00, 3.45278450e+01,
       1.15092817e+01, 6.04237288e+00, 1.61129944e+02, 3.22259887e+01,
       2.54415700e+01, 1.33828857e-01, 1.09861325e+01, 5.43134641e+00,
       1.61129944e+02, 8.05649718e+01, 1.01766280e+00, 5.14244501e+00,
       7.79661017e+00, 2.96558178e+00, 9.18992073e-01, 6.04237288e+00,
       4.83389831e+02, 1.41342056e+00, 8.05649718e+01, 1.18188223e+00,
       3.71838331e+01, 2.84346959e+01, 4.39445300e+01, 2.54415700e+01,
       2.62711864e+00, 6.04237288e+01, 2.00576693e+00, 1.32799404e+00,
       1.49656294e+00, 8.05649718e+01, 4.83389831e+02, 2.87732042e+00,
       8.04309202e-01, 2.01412429e+01, 4.83389831e+02, 1.20847458e+02,
       2.40492453e+00, 4.20704813e-01, 2.67806000e-01, 8.05649718e+01,
      

In [30]:
np.unique(codes_list)

array([        4557003,         5602001,         6525002,        10509002,
              15802004,        22298006,        26929004,        27942005,
              36971009,        39898005,        40055000,        40095003,
              43878008,        44054006,        46177005,        47505003,
              48724000,        49436004,        49915006,        53827007,
              56786000,        59621000,        60234000,        60573004,
              61977001,        62479008,        64859006,        65710008,
              67782005,        73430006,        75498004,        76571007,
              76916001,        78275009,        79619009,        80394007,
              84114007,        84757009,        86175003,        86406008,
              87433001,        87628006,        88805009,        90460009,
              91302008,        91434003,        91861009,        92691004,
             110030002,       110359009,       111282000,       111287006,
             125601008,  